In [1]:
%cd ../../

/home/PJWSTK/s14028/Engine


In [2]:
import keras

from common import *
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v1 import CNN_v1

Using TensorFlow backend.


In [3]:
train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

In [4]:
cnn = CNN_v1((480, 640, 3), split_into_parts=40)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v1/weights/"

In [7]:
model_names = ["cnn_v1_40_1e_0_100",
               "cnn_v1_40_1e_3_50",
               "cnn_v1_40_1e_3_100",
               "cnn_v1_40_1e_3_150"]

model_names = [prefix + name for name in model_names]

In [8]:
models = [CNN_v1((480, 640, 3), split_into_parts=40) for i in range(len(model_names))]

In [9]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=3, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.6934 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.6934 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.6935 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.6934 - acc: 1.0000


In [10]:
for name, result in results.items():
  diff = best_cop_diff(result, count_test)
  
  print(f"Model {name} behaved:")
  
  print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
  print(diff[2])
  
  print(f"With MSE {mse(diff[2])}")
  print(f"With MAE {mae(diff[2])}", end="\n\n")

Model CNN/CNN_v1/weights/cnn_v1_40_1e_0_100 behaved:
For cut-of-point 0.2 had accuracy 84.47549557460538:
[  7  -3   3   7  12   6  -2  -3   4  -7  -6  -2  -5  -6  -1  -3   6   0
  -2   3  -5   0  -3  -9   0  -4  -7   1  -1  -4   9  -2  -2   7   9  14
  14   9   7   8   8   2  16   1   0   1  14   5   3   1   2   1  -8  -7
  -5   2   2   7   2  -1  19  -3   6   4   6  -3  -7 -12 -14   7  -4  -5
   3  -1  -2   1  -7  -7  -5  -7  -3  -4  -1  -5  -1  -4  -3  -1   6   1
  -1   2   3 -19   1 -10  -4  -7  -5   3   3  -4   8  -5   5   4  -3  -5
  -7   2   1  -1   0   1  -1  12  10   2   1  -5  -7  -1   4   0  -9  -5
  -3  -6  -4  -3   4   3 -13  -8   8  -1   0  -9 -10   4   1  -3  -3   1
   6   5   4   5   2  -3  -2  -3   4   0  -3  -1   4   1   3   5   3  -1
   3   0  -3  14   0   4  -4   2   3   1  -5   5   4  -3   3   3   9   8
  13  -3  -7  -1   1   3  -8   0   6  -1   0   2  -2  -7  -5   4  -3   3
   2   3  -1   5   5   1   9   6   6  -3   4   0   2   9  -3  -4   3  10
  12   6   6   6  